# <center>Introduction to ML - Decision Tree Coursework</center>
### <center>COMP70050</center>

### Installing dependencies

Step 1 

In [1]:
import numpy as np

clean_data = x = np.loadtxt("wifi_db/clean_dataset.txt", delimiter='\t')
noisy_data = x = np.loadtxt("wifi_db/noisy_dataset.txt")

X_clean = clean_data[:, :-1]
y_clean = clean_data[:, -1]

X_noisy = noisy_data[:, :-1]
y_noisy = noisy_data[:, -1]

In [2]:
def entropy(labels):
    _, counts = np.unique(labels, return_counts=True)
    prob = counts / np.sum(counts)
    return -np.sum(prob * np.log2(prob))

In [3]:
def find_split(dataset):
        # We split out dataset, x is the array of features and y is the labels
        x = dataset[:, :-1]
        y = dataset[:, -1]

        # n is the number of samples and k is the number of features
        n, k = x.shape

        H_total = entropy(y)
    
        max_gain = 0
        best_attribute = -1
        best_split = None

        for attribute in range(k):
            arr = x[:, attribute]
            uniques = np.unique(arr)

            for split in uniques:
                left_ds = y[arr <= split]
                right_ds = y[arr > split]

                remainder = ((len(left_ds)/len(arr)) * entropy(left_ds)) + ((len(right_ds)/len(arr)) * entropy(right_ds))
                
                if max_gain < H_total - remainder:

                    max_gain = H_total - remainder
                    best_attribute = attribute
                    best_split = split
        
        return best_attribute, best_split

In [4]:
def decision_tree_learning(dataset, depth=0):
    total_instances = len(dataset)
    x = dataset[:, :-1]
    y = dataset[:, -1]
    
    if len(np.unique(y)) == 1:
        return (Node(None, y[0], None, None, total_instances, True), depth)
    else:
        attribute, value = find_split(dataset)
        l_dataset = dataset[dataset[:, attribute] <= value]
        r_dataset = dataset[dataset[:, attribute]  > value]
        l_node, l_depth = decision_tree_learning(l_dataset, depth + 1)
        r_node, r_depth = decision_tree_learning(r_dataset, depth + 1)
        node = Node(attribute, value, l_node, r_node, total_instances, False)
        return (node, max(l_depth, r_depth))

In [5]:
class Node:
    def __init__(self, attribute, value, left, right, n_instances: int, is_leaf: bool):
        self.attribute = attribute
        self.value = value
        self.left = left
        self.right = right
        self.n_instances = n_instances
        self.is_leaf = is_leaf


In [6]:
def predict(node, X):
    if node.is_leaf:
        return np.full(len(X), node.value)
    
    left_indices = X[:, node.attribute] <= node.value
    right_indices = X[:, node.attribute] > node.value
    
    predictions  = np.zeros(len(X))
    predictions[left_indices] = predict(node.left, X[left_indices])
    predictions[right_indices] = predict(node.right, X[right_indices])

    return predictions  

### Step 3 : Evaluation

In [7]:
def confusion_matrix(y_gold, y_prediction, class_labels=None):

    # if no class_labels are given, we obtain the set of unique class labels from
    # the union of the ground truth annotation and the prediction
    if not class_labels:
        class_labels = np.unique(np.concatenate((y_gold, y_prediction)))

    confusion = np.zeros((len(class_labels), len(class_labels)), dtype=np.int8)

    # for each correct class (row),
    # compute how many instances are predicted for each class (columns)
    for i, correct_class in enumerate(class_labels):
        for j, predicted_class in enumerate(class_labels):
            count = np.count_nonzero(np.logical_and(y_gold == correct_class, y_prediction == predicted_class))
            confusion[i][j] = count

    return confusion

def accuracy(confusion):
    return np.trace(confusion) / np.sum(confusion) if np.sum(confusion) > 0 else 0


def precision(confusion):

    # Compute the precision per class
    p = np.diag(confusion) / np.sum(confusion, axis=1)

    # Compute the macro-averaged precision
    macro_p = np.mean(p)

    return (p, macro_p)

def recall(confusion):

    # Compute the recall per class
    r = np.diag(confusion) / np.sum(confusion, axis=0)
    # Compute the macro-averaged recall
    macro_r = np.mean(r)
    return (r, macro_r)

def f1_score(confusion):


    (precisions, macro_p) = precision(confusion)
    (recalls, macro_r) = recall(confusion)

    # just to make sure they are of the same length
    assert len(precisions) == len(recalls)

    f = (2*precisions*recalls) / (precisions + recalls)

    macro_f = np.mean(f)

    return (f, macro_f)

In [8]:
def train_test_k_fold(n_folds, n_instances, random_generator=np.random.default_rng()):

    shuffled_indices = random_generator.permutation(n_instances)
    split_indices = np.array_split(shuffled_indices, n_folds)

    folds = []
    for k in range(n_folds):
        test_indices = split_indices[k]
        train_indices = np.concatenate(split_indices[:k] + split_indices[k+1:])

        folds.append([train_indices, test_indices])

    return folds

### Step 4 : Pruning

In [9]:
def evaluate(test_dataset, trained_tree):
    y_pred = predict(trained_tree, test_dataset[:, :-1])
    
    accuracy = np.mean(y_pred == test_dataset[:, -1])
    return accuracy

In [10]:
def prune_tree(node: Node, train_dataset, val_dataset):
    
    if node.is_leaf:
        return node

    left_train_dataset = train_dataset[train_dataset[:, node.attribute] <= node.value]
    right_train_dataset = train_dataset[train_dataset[:, node.attribute] > node.value]
    
    node.left = prune_tree(node.left, left_train_dataset, val_dataset)
    node.right = prune_tree(node.right, right_train_dataset, val_dataset)
    
    if node.left.is_leaf and node.right.is_leaf:

        new_value = node.left.value if node.left.n_instances > node.right.n_instances else node.right.value
        potential_leaf = Node(None, new_value, None, None, len(train_dataset), True)

        if evaluate(val_dataset, node) <= evaluate(val_dataset, potential_leaf):
            return potential_leaf

    return node


In [11]:
dataset = noisy_data
n_folds = 10
n_instances = len(dataset)

confusion_matrices = np.zeros((n_folds, 4, 4))
accuracies = np.zeros(n_folds)
precisions = np.zeros(n_folds)
recalls = np.zeros(n_folds)
f1_scores = np.zeros(n_folds)

confusion_matrices_after = np.zeros((n_folds, 4, 4))
accuracies_after = np.zeros(n_folds)
precisions_after = np.zeros(n_folds)
recalls_after = np.zeros(n_folds)
f1_scores_after = np.zeros(n_folds)

for i, (train_indices, test_indices) in enumerate(train_test_k_fold(n_folds, len(x))):
    # Splitting the train and test
    x_train = dataset[train_indices, :-1]
    y_train = dataset[train_indices, -1]
    x_test = dataset[test_indices, :-1]
    y_test = dataset[test_indices, -1]

    root, depth = decision_tree_learning(dataset[train_indices])
    y_pred = predict(root, x_test)

    root = prune_tree(root, dataset[train_indices], dataset[test_indices])
    y_pred_after = predict(root, x_test)

    confusion_matrices[i] = confusion_matrix(np.int8(y_test), np.int8(y_pred))
    accuracies[i] = accuracy(confusion_matrices[i])
    precisions[i] = precision(confusion_matrices[i])[1]
    recalls[i] = recall(confusion_matrices[i])[1]
    f1_scores[i] = f1_score(confusion_matrices[i])[1]
    train_dataset = np.column_stack((x_test, y_test))    

    confusion_matrices_after[i] = confusion_matrix(np.int8(y_test), np.int8(y_pred_after))
    accuracies_after[i] = accuracy(confusion_matrices_after[i])
    precisions_after[i] = precision(confusion_matrices_after[i])[1]
    recalls_after[i] = recall(confusion_matrices_after[i])[1]
    f1_scores_after[i] = f1_score(confusion_matrices_after[i])[1]
    train_dataset_after = np.column_stack((x_test, y_test))  
    
print("Before pruning:")
print(f"Accuracy: {accuracies.mean()}")
print(f"Precision: {precisions.mean()}")
print(f"Recall: {recalls.mean()}")
print(f"F1 Score: {f1_scores.mean()}")

print("\nAfter pruning:")
print(f"Accuracy: {accuracies_after.mean()}")
print(f"Precision: {precisions_after.mean()}")
print(f"Recall: {recalls_after.mean()}")
print(f"F1 Score: {f1_scores_after.mean()}")


Before pruning:
Accuracy: 0.7979999999999999
Precision: 0.7978295300553822
Recall: 0.7996149848876579
F1 Score: 0.7970156093650821

After pruning:
Accuracy: 0.8230000000000001
Precision: 0.8225623770802253
Recall: 0.8248185242951935
F1 Score: 0.8222422039576095
